In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from scipy.spatial import distance

In [ ]:
try:
    # Tạo một model đơn giản
    m = gp.Model("test_model")

    # Tạo biến x và y
    x = m.addVar(name="x", vtype=GRB.BINARY)
    y = m.addVar(name="y", vtype=GRB.CONTINUOUS)

    # Đặt hàm mục tiêu: Maximize x + y
    m.setObjective(x + y, GRB.MAXIMIZE)

    # Thêm ràng buộc: x + y <= 1.5
    m.addConstr(x + y <= 1.5, "c0")

    # Tối ưu hóa
    m.optimize()

    print(f"\nKết quả tối ưu: x={x.X}, y={y.X}")
    print("Chúc mừng! Bạn đã cài đặt và kết nối License thành công.")

except gp.GurobiError as e:
    print(f"Lỗi Gurobi: {e}")
except Exception as e:
    print(f"Lỗi khác: {e}")

In [2]:
class VRPSDP:
    """
    Lớp giải quyết bài toán Vehicle Routing Problem with Simultaneous Delivery and Pick-up (VRPSDP)
    sử dụng Gurobi.
    
    Mô hình toán học được dựa trên công thức vehicle-flow của Rieck and Zimmermann (2013),
    mục 4.2.
    """
    def __init__(self, costMatrix, demand, pickup, numberOfVehicles, capacityOfVehicle):
        self.costMatrix = costMatrix
        self.n = len(costMatrix)
        self.nodes = range(self.n)
        self.customers = range(1, self.n)
        
        self.demand = demand
        self.pickup = pickup
        self.numberOfVehicles = numberOfVehicles
        self.capacityOfVehicle = capacityOfVehicle
        
        # Biến để lưu trữ mô hình và các biến quyết định
        self.model = None
        self.x = None
        self.l = None
        self.ld = None
        
        # Khởi tạo mô hình Gurobi
        self._initialize_model()

    def _initialize_model(self):
        """
        Hàm nội bộ để thiết lập mô hình Gurobi, bao gồm các biến, hàm mục tiêu và ràng buộc.
        """
        # 1. Khởi tạo mô hình
        self.model = gp.Model('VRPSDP_Gurobi')
        
        # 2. Định nghĩa các biến
        # x[i, j] = 1 nếu xe đi từ i đến j, 0 nếu ngược lại
        self.x = self.model.addVars(self.nodes, self.nodes, vtype=GRB.BINARY, name="x")

        # l[i] = tải trọng trên xe sau khi thăm khách hàng i
        self.l = self.model.addVars(self.customers, vtype=GRB.CONTINUOUS, name="l")

        # ld[i] = tổng lượng hàng cần giao cho khách hàng i và tất cả các khách hàng sau đó
        self.ld = self.model.addVars(self.nodes, vtype=GRB.CONTINUOUS, name="ld")

        # Hằng số Big-M cho các ràng buộc
        # Một lựa chọn an toàn cho M là tổng tất cả nhu cầu hoặc tải trọng xe
        M1 = self.capacityOfVehicle
        M2 = self.capacityOfVehicle # Giá trị an toàn cho ràng buộc tải trọng
        
        # 3. Thiết lập hàm mục tiêu (Công thức 2)
        objective = gp.quicksum(self.costMatrix[i][j] * self.x[i, j] 
                                for i in self.nodes for j in self.nodes if i != j)
        self.model.setObjective(objective, GRB.MINIMIZE)

        # 4. Thêm các ràng buộc
        
        # Ràng buộc cơ bản: không có cạnh nào đi vào chính nó
        self.model.addConstrs((self.x[i, i] == 0 for i in self.nodes), name="no_self_loops")

        # Ràng buộc (3) & (4): Mỗi khách hàng được thăm đúng một lần
        self.model.addConstrs((gp.quicksum(self.x[i, j] for i in self.nodes if i != j) == 1 
                               for j in self.customers), name="indegree")
        self.model.addConstrs((gp.quicksum(self.x[j, i] for i in self.nodes if i != j) == 1 
                               for j in self.customers), name="outdegree")

        # Ràng buộc (5): Số lượng xe sử dụng không vượt quá số xe có sẵn
        self.model.addConstr(gp.quicksum(self.x[0, j] for j in self.customers) <= self.numberOfVehicles, 
                             name="num_vehicles")

        # Ràng buộc (6): Dòng chảy của lượng hàng cần giao và loại bỏ tuyến đường con
        self.model.addConstrs((self.ld[i] >= self.ld[j] + self.demand[i] - M1 * (1 - self.x[i, j])
                               for i in self.nodes for j in self.customers if i != j), name="delivery_flow")

        # Ràng buộc (7): Tính tải trọng cho khách hàng đầu tiên trong một tuyến
        self.model.addConstrs((self.l[j] >= self.ld[j] - self.demand[j] + self.pickup[j] 
                               for j in self.customers), name="load_first_customer")

        # Ràng buộc (8): Tính tải trọng cho các khách hàng tiếp theo
        self.model.addConstrs((self.l[j] >= self.l[i] - self.demand[j] + self.pickup[j] - M2 * (1 - self.x[i, j])
                               for i in self.customers for j in self.customers if i != j), name="load_other_customers")

        # Ràng buộc (9): Giới hạn cho biến ld (lượng hàng cần giao)
        self.model.addConstrs((self.ld[i] <= self.capacityOfVehicle for i in self.nodes), name="ld_upper_bound")
        self.model.addConstrs((self.ld[i] >= self.demand[i] for i in self.nodes), name="ld_lower_bound")

        # Ràng buộc (10): Giới hạn cho biến l (tải trọng vật lý trên xe)
        self.model.addConstrs((self.l[i] <= self.capacityOfVehicle for i in self.customers), name="l_upper_bound")
        self.model.addConstrs((self.l[i] >= self.pickup[i] for i in self.customers), name="l_lower_bound")
        
        # Ràng buộc (11) đã được định nghĩa khi khai báo biến x là BINARY

    def solve(self, time_limit=None):
        """
        Giải mô hình VRPSDP.
        :param time_limit: Giới hạn thời gian giải (tính bằng giây).
        """
        print("Bắt đầu giải bài toán VRPSDP bằng Gurobi...")
        if time_limit:
            self.model.setParam(GRB.Param.TimeLimit, time_limit)
        
        self.model.optimize()

    def getResult(self):
        """
        Trích xuất và hiển thị kết quả sau khi giải.
        """
        if self.model.solCount > 0:
            print("\n--------------------------------------------------")
            print(f"Đã tìm thấy lời giải tối ưu!")
            print(f"Tổng chi phí: {self.model.ObjVal:.2f}")
            
            # Trích xuất các tuyến đường
            solution_edges = []
            for i in self.nodes:
                for j in self.nodes:
                    if i != j and self.x[i, j].X > 0.5:
                        solution_edges.append((i, j))
            
            # Xây dựng lại các tuyến đường từ các cạnh
            routes = []
            starts = [j for i, j in solution_edges if i == 0]
            for start_node in starts:
                route = [0, start_node]
                current_node = start_node
                while current_node != 0:
                    # Tìm cạnh tiếp theo
                    found = False
                    for i, j in solution_edges:
                        if i == current_node:
                            route.append(j)
                            current_node = j
                            found = True
                            break
                    if not found: # Đề phòng lỗi vô tận
                        print("Lỗi: Không thể xây dựng lại tuyến đường hoàn chỉnh.")
                        break
                routes.append(route)
            
            print(f"Số tuyến đường: {len(routes)}")
            for i, route in enumerate(routes):
                route_cost = 0
                for k in range(len(route) - 1):
                    u, v = route[k], route[k+1]
                    route_cost += self.costMatrix[u][v]
                print(f"  Tuyến {i+1}: {' -> '.join(map(str, route))} (Chi phí: {route_cost})")

            print("--------------------------------------------------")
        elif self.model.status == GRB.INFEASIBLE:
            print("Mô hình không khả thi (infeasible). Vui lòng kiểm tra lại các ràng buộc.")
        else:
            print("Không tìm thấy lời giải tối ưu trong thời gian cho phép.")

# --- Dữ liệu đầu vào ---
costMatrix = [[0,9,14,23,32,50,21,49,30,27,35,28,18],   
              [9,0,21,22,36,52,24,51,36,37,41,30,20],    
              [14,21,0,25,38,5,31,7,36,43,29,7,6],    
              [23,22,25,0,42,12,35,17,44,31,31,11,6],
              [32,36,38,42,0,22,37,16,46,37,29,13,14],   
              [50,52,5,12,22,0,41,23,10,39,9,17,16],   
              [21,24,31,35,37,41,0,26,21,19,10,25,12],  
              [49,51,7,17,16,23,26,0,30,28,16,27,12],   
              [30,36,36,44,46,10,21,30,0,25,22,10,20],    
              [27,37,43,31,37,39,19,28,25,0,20,16,8],   
              [35,41,29,31,29,9,10,16,22,20,0,10,10],   
              [28,30,7,11,13,17,25,27,10,16,10,0,10],
              [18,20, 6, 6,14,16,12,12,20,8, 10,10,0]]

demand = [0, 1200, 1700, 1500, 1400, 1700, 1400, 1200, 1900, 1800, 1600, 1700, 1100] 
# Lưu ý: pickup tại kho phải bằng tổng demand của khách hàng, và demand tại kho bằng tổng pickup của khách hàng.
# Dữ liệu của bạn có pickup[0]=1100, điều này có thể làm bài toán infeasible.
# Tôi sẽ tạm chỉnh lại pickup[0]=0 để mô hình có thể chạy.
pickup = [0, 0, 1200, 1700, 1500, 1400, 1700, 1400, 1200, 1900, 1800, 1600, 1700]

# Chỉnh lại pickup để có cùng độ dài với costMatrix
# pickup.insert(1, 0) # Thêm pickup = 0 cho khách hàng 1, theo dữ liệu gốc của bạn
# pickup[0] = sum(pickup) # pickup tại kho = tổng pickup khách hàng
# demand[0] = sum(demand) # demand tại kho = tổng demand khách hàng

capacityOfVehicle = 6000
numberOfVehicles = 4

# --- Sử dụng lớp ---
lp2 = VRPSDP(costMatrix, demand, pickup, numberOfVehicles, capacityOfVehicle)
lp2.solve() # Giới hạn thời gian chạy là 60 giây
lp2.getResult()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2739981
Academic license 2739981 - for non-commercial use only - registered to so___@sis.hust.edu.vn
Bắt đầu giải bài toán VRPSDP bằng Gurobi...
Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-6300U CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Academic license 2739981 - for non-commercial use only - registered to so___@sis.hust.edu.vn
Optimize a model with 376 rows, 194 columns and 1215 nonzeros (Min)
Model fingerprint: 0xc77ed945
Model has 156 linear objective coefficients
Variable types: 25 continuous, 169 integer (169 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+03]
  Objective range  [5e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+03]
Presolve removed 75 rows and 14 columns
Presolve time: 0.02s
Presolved: 301 

In [44]:
xCoordinates = [145, 151, 159, 130, 128, 163, 146, 161, 142, 163, 148, 128, 156, 129, 146, 164, 141, 147, 164, 129, 155, 139]
yCoordinates = [215, 164, 261, 254, 252, 247, 246, 242, 239, 236, 232, 231, 217, 214, 208, 208, 206, 193, 193, 189, 185, 182]

costMatrix = np.ndarray(shape=(len(xCoordinates), len(yCoordinates)))
for i in range(len(xCoordinates)):
    for j in range(len(yCoordinates)):
        costMatrix[i][j] = float(distance.euclidean([xCoordinates[i],yCoordinates[i]], [xCoordinates[j],yCoordinates[j]]))

demand = [0, 1100, 700, 800, 1400, 2100, 400, 800, 100, 500, 600, 1200, 1300, 1300, 300, 900, 2100, 1000, 900, 2100, 1000, 900]
pickup = [0, 0, 1100, 700, 800, 1400, 2100, 400, 800, 100, 500, 600, 1200, 1300, 1300, 300, 900, 2100, 1000, 900, 2100, 1000]
capacityOfVehicle = 6000
numberOfVehicles = 7

In [ ]:
lp2 = VRPSDP(costMatrix, demand, pickup, numberOfVehicles, capacityOfVehicle)
lp2.solve() # Giới hạn thời gian chạy là 60 giây
lp2.getResult()

In [3]:
xCoordinates = [162, 218, 218, 201, 214, 224, 210, 104, 126, 119, 129, 126, 125, 116, 126, 125, 119, 115, 153, 175, 180, 159, 188, 152, 215, 212, 188, 207, 184, 207]
yCoordinates = [354, 382, 358, 370, 371, 370, 382, 354, 338, 340, 349, 347, 346, 355, 335, 355, 357, 341, 351, 363, 360, 331, 357, 349, 389, 394, 393, 406, 410, 392]

costMatrix = np.ndarray(shape=(len(xCoordinates), len(yCoordinates)))
for i in range(len(xCoordinates)):
    for j in range(len(yCoordinates)):
        costMatrix[i][j] = float(distance.euclidean([xCoordinates[i],yCoordinates[i]], [xCoordinates[j],yCoordinates[j]]))

demand = [0, 300, 3100, 125, 100, 200, 150, 150, 450, 300, 100, 950, 125, 150, 150, 550, 150, 100, 150, 400, 300, 1500, 100, 300, 500, 800, 300, 100, 150, 1000]
pickup = [0, 0, 300, 3100, 125, 100, 200, 150, 150, 450, 300, 100, 950, 125, 150, 150, 550, 150, 100, 150, 400, 300, 1500, 100, 300, 500, 800, 300, 100, 150]
capacityOfVehicle = 4500
numberOfVehicles = 4

lp2 = VRPSDP(costMatrix, demand, pickup, numberOfVehicles, capacityOfVehicle)
lp2.solve()
lp2.getResult()

Bắt đầu giải bài toán VRPSDP bằng Gurobi...
Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-6300U CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Academic license 2739981 - for non-commercial use only - registered to so___@sis.hust.edu.vn
Optimize a model with 1889 rows, 959 columns and 6876 nonzeros (Min)
Model fingerprint: 0x3624f5ed
Model has 870 linear objective coefficients
Variable types: 59 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+03]
Presolve removed 191 rows and 38 columns
Presolve time: 0.09s
Presolved: 1698 rows, 921 columns, 13179 nonzeros
Variable types: 58 continuous, 863 integer (863 binary)

Root relaxation: objective 2.817909e+02, 128 iterations, 0.01 seconds (0.00 work units)

 